## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np

### Ocultando mensagens de avisos no Pandas:

In [68]:
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Importando os arquivos .csv e atribuindo a variáveis:

In [37]:
d_2019 = pd.read_csv('Dados_csv/Ano-2019.csv', sep=';')
d_2020 = pd.read_csv('Dados_csv/Ano-2020.csv', sep=';')
d_2021 = pd.read_csv('Dados_csv/Ano-2021.csv', sep=';')

print('O dataset de 2019 tem', d_2019.shape[0], 'linhas e', d_2019.shape[1], 'colunas.\n'
      'O dataset de 2020 tem', d_2020.shape[0], 'linhas e', d_2020.shape[1], 'colunas.\n'
      'O dataset de 2021 tem', d_2021.shape[0], 'linhas e', d_2021.shape[1], 'colunas.')

O dataset de 2019 tem 288992 linhas e 31 colunas.
O dataset de 2020 tem 160903 linhas e 31 colunas.
O dataset de 2021 tem 105605 linhas e 31 colunas.


## Unificando as bases 2019, 2020 e 2021 em um único DataFrame:

In [39]:
d_unificado = pd.concat([d_2019,d_2020,d_2021])

print('O dataframe unificado possui', d_unificado.shape[0], 'linhas e', d_unificado.shape[1], 'colunas')

O dataframe unificado possui 555500 linhas e 31 colunas


## Verificando os nomes das colunas:

In [22]:
d_unificado.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'vlrRestituicao', 'nuDeputadoId', 'ideDocumento',
       'urlDocumento'],
      dtype='object')

   ><b>OBS:</b>A formatação do nome das colunas está no padrão "CamelCase"

## Checando o espaço físico do dataset unificado:

In [44]:
tamanho = sum(d_unificado.memory_usage())

print('O espaço físico ocupado em MB pelo dataset unificado é de', tamanho/1000000, 'MB')

O espaço físico ocupado em MB pelo dataset unificado é de 142.208 MB


## Checando os tipos de formatos em todas as colunas:

In [45]:
d_unificado.dtypes

txNomeParlamentar             object
cpf                          float64
ideCadastro                  float64
nuCarteiraParlamentar        float64
nuLegislatura                  int64
sgUF                          object
sgPartido                     object
codLegislatura                 int64
numSubCota                     int64
txtDescricao                  object
numEspecificacaoSubCota        int64
txtDescricaoEspecificacao     object
txtFornecedor                 object
txtCNPJCPF                    object
txtNumero                     object
indTipoDocumento               int64
datEmissao                    object
vlrDocumento                 float64
vlrGlosa                     float64
vlrLiquido                   float64
numMes                         int64
numAno                         int64
numParcela                     int64
txtPassageiro                 object
txtTrecho                     object
numLote                        int64
numRessarcimento             float64
v

><b>OBSERVAÇÕES:</b>
>>1. Verificado que a coluna 'datEmissao' está no formato <b>'Object'</b>, a mesma deverá ser convertida para o formato de data.
>>2. Verificado que as colunas que contém os valores das despesas já estão em formato <b>'float64'</b>, dispensando assim sua conversão, salvo se a formatação do separador de centavos estiver fora do padrão americano (ponto).

## Verificando o percentual de d_copia nulos ('NaN') no dataset unificado:

>Nessa etapa o .isnull() foi utilizado, acompanhado do .mean() para encontrar o percentual de d_copia faltantes em cada coluna deste dataset.

In [52]:
percentual_nulos = pd.DataFrame(round(d_unificado.isnull().mean()*100,2)) # Número de dados nulos em percentual

percentual_nulos.rename(columns={0:'Percentual de Dados Nulos (%)'}) 

,Percentual de Dados Nulos (%)
txNomeParlamentar,0.00
cpf,0.33
ideCadastro,0.31
nuCarteiraParlamentar,0.31
nuLegislatura,0.00
sgUF,0.31
sgPartido,0.31
codLegislatura,0.00
numSubCota,0.00
txtDescricao,0.00


><b>OBSERVAÇÕES:</b>
>>Verificado altos índices de d_copia nulos (faltantes) nas seguintes colunas:
    >>><b>vlrRestituicao (99,93%)</b>    
    >>><b>txtTrecho (76,88%)</b>     
    >>><b>txtPassageiro (76,69%)</b>    
    >>><b>txtDescricaoEspecificacao (75,49%)</b>    
    >>><b>urlDocumento (37,06%)</b>

>>No entanto, nas colunas que se referem aos valores financeiros o índice de d_copia nulos é aceitável.

# Tratamento e Limpeza dos Dados

### Configurando o Pandas para exibição de todas as 31 colunas do dataset:

In [54]:
pd.set_option('display.max_columns',31) # Setando o Pandas para exibir todas as 31 colunas

### Convertendo a coluna 'datEmissao' do tipo 'Object' para o tipo 'Datetime' em todos as fontes de dados:

In [55]:
d_2019['datEmissao'] = pd.to_datetime(d_2019['datEmissao'], format = '%Y/%m/%d')

d_2020['datEmissao'] = pd.to_datetime(d_2020['datEmissao'], format = '%Y/%m/%d')

d_2021['datEmissao'] = pd.to_datetime(d_2021['datEmissao'], format = '%Y/%m/%d')

d_unificado['datEmissao'] = pd.to_datetime(d_unificado['datEmissao'], format = '%Y/%m/%d')

### Criando uma cópia do dataset como Boa Prática de Programação:

In [59]:
d_copia = d_unificado

### Verificando a existência de despesas sem atribuição do partido político:

In [61]:
sem_partido = d_copia.query('sgPartido.isnull()')

### Verificado após uma amostragem que os lançamentos sem partidos políticos, se referem a lançamentos de suas lideranças partidárias.

In [69]:
sem_partido.sample(5)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
856,LIDERANÇA DO SOLIDARIEDADE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SILT SELF SERVICE EIRELI ME,072.867.310/0019-4,499,4,2019-09-05 00:00:00,300.00,0.0,300.00,8,2019,0,NaN,NaN,1630260,NaN,NaN,2864,6911657,http://camara.leg.br/cota-parlamentar/nota-fis...
650,LIDERANÇA DO PT,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,M.E. CARDOSO RESTAURANTE E LANCHONETE,090.170.420/0010-0,333,0,2019-12-03 00:00:00,255.00,0.0,255.00,12,2019,0,NaN,NaN,1659275,NaN,NaN,2439,6977981,https://www.camara.leg.br/cota-parlamentar/doc...
230,LIDERANÇA DO PT,NaN,NaN,NaN,2019,NaN,NaN,56,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,POSTO DE COMBUSTIVEL GUERREIRINHO LTDA,353.405.290/0013-9,18958,4,2020-11-18 01:00:00,168.43,0.0,168.43,11,2020,0,NaN,NaN,1728871,NaN,NaN,2439,7120237,http://camara.leg.br/cota-parlamentar/nota-fis...
324,LIDERANÇA DO PTB,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,MERCADINHO LA PALMA LTDA,006.726.000/0010-4,23957,4,2020-03-03 00:00:00,270.70,0.0,270.70,3,2020,0,NaN,NaN,1684836,NaN,NaN,2722,7032968,http://camara.leg.br/cota-parlamentar/nota-fis...
14,LIDERANÇA DO CIDADANIA,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,ARTESANAL SERVICOS DE ALIMENTACAO E BUFFET EIRELI,228.362.100/0015-7,213,4,2021-07-07 00:00:00,720.00,0.0,720.00,7,2021,0,NaN,NaN,1779394,NaN,NaN,3453,7223049,http://camara.leg.br/cota-parlamentar/nota-fis...


### Obtendo os nomes únicos das Lideranças Partidárias:

In [66]:
lista_liderancas = sem_partido['txNomeParlamentar'].value_counts()

lista_liderancas = lista_liderancas.reset_index()

lista_liderancas = pd.DataFrame(lista_liderancas)

partidos_completo = lista_liderancas['index'].str.split()

partidos = []

for i in partidos_completo:
    partidos.append(i[-1])
    
lista_liderancas['sgPartido'] = partidos

del(lista_liderancas['txNomeParlamentar'])

lista_liderancas

,index,sgPartido
0,LIDERANÇA DO PT,PT
1,LIDERANÇA DO PTB,PTB
2,LIDERANÇA DO PSDB,PSDB
3,LIDERANÇA DO PODEMOS,PODEMOS
4,LIDERANÇA DO SOLIDARIEDADE,SOLIDARIEDADE
5,LIDERANÇA DO PSD,PSD
6,LIDERANÇA DO NOVO,NOVO
7,LIDERANÇA DO DEMOCRATAS,DEMOCRATAS
8,LIDERANÇA DO CIDADANIA,CIDADANIA
9,LIDMIN,LIDMIN


### Atribuindo as Lideranças Partidárias aos seus respectivos partidos políticos:

In [71]:
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSL", 'PSL', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PT", 'PT', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PTB", 'PTB', d_copia.sgPartido) 
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSDB", 'PSDB', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PODEMOS", 'PODEMOS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO SOLIDARIEDADE", 'SOLIDARIEDADE', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSD", 'PSD', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO NOVO", 'NOVO', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO DEMOCRATAS", 'DEMOCRATAS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO CIDADANIA", 'CIDADANIA', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO AVANTE", 'AVANTE', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PDT", 'PDT', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSOL", 'PSOL', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PROGRESSISTAS", 'PROGRESSISTAS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL", 'REPUBLICANOS', d_copia.sgPartido)

### Checando os dados das Lideranças que ficaram sem Partidos:

In [80]:
d_copia[d_copia['sgPartido'].isnull()]['txNomeParlamentar'].unique()

array(['LID.GOV-CD', 'LIDMIN'], dtype=object)

### Filtrando apenas os valores Positivos na coluna 'vlrLiquido'

In [82]:
valor_positivo = d_copia.query('vlrLiquido > 0')

### Filtrando apenas os lançamentos do Estado do Mato Grosso (MT):

In [89]:
dados_mt = valor_positivo.query('sgUF == "MT"')

dados_mt.sample(10)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
78164,Nelson Barbudo,1.862798e+09,204449.0,405.0,2019,MT,PSL,56,9,PASSAGEM AÉREA - REEMBOLSO,0,NaN,LATAM,020.128.620/0016-0,9572149776430,0,2021-06-27 00:00:00,906.03,0.00,906.03,6,2021,0,NaN,NaN,1776958,NaN,NaN,3388,7217874,https://www.camara.leg.br/cota-parlamentar/doc...
175059,Emanuel Pinheiro Neto,5.585742e+08,198783.0,403.0,2019,MT,PTB,56,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,America Auto Posto Ltda F01,369.335.390/0014-0,57649,0,2019-07-17 00:00:00,50.00,0.00,50.00,7,2019,0,NaN,NaN,1618594,NaN,NaN,3315,6884761,https://www.camara.leg.br/cota-parlamentar/doc...
118383,Nelson Barbudo,1.862798e+09,204449.0,405.0,2019,MT,PSL,56,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,BOM CLIMA COM DE COMB E LUBRIF LTDA,028.696.150/0018-4,880769,0,2020-01-25 00:00:00,265.02,0.00,265.02,1,2020,0,NaN,NaN,1669677,NaN,NaN,3388,6999835,https://www.camara.leg.br/cota-parlamentar/doc...
83271,Juarez Costa,4.784308e+10,204474.0,401.0,2019,MT,MDB,56,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,AUTO POSTO 306 NORTE LTDA,223.988.180/0014-7,147091,4,2021-06-07 00:00:00,161.96,0.00,161.96,6,2021,0,NaN,NaN,1771999,NaN,NaN,3237,7208723,http://camara.leg.br/cota-parlamentar/nota-fis...
82301,José Medeiros,4.244254e+10,204472.0,404.0,2019,MT,PODE,56,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,MAXPEL+CAR alexei maxwell,339.528.100/0010-5,73,4,2021-02-27 00:00:00,148.50,0.00,148.50,2,2021,0,NaN,NaN,1748659,NaN,NaN,3411,7160113,http://camara.leg.br/cota-parlamentar/nota-fis...
26856,Valtenir Pereira,3.435810e+10,141552.0,536.0,2019,MT,MDB,56,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0,NaN,Rota do Oeste,195.213.220/0010-4,12394,1,2021-05-23 00:00:00,4.10,0.00,4.10,5,2021,0,NaN,NaN,1768831,NaN,NaN,1946,7201886,https://www.camara.leg.br/cota-parlamentar/doc...
231920,José Medeiros,4.244254e+10,204472.0,404.0,2019,MT,PODE,56,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0,NaN,UBER DO BRASIL TECNOLOGIA LTDA.,178.956.460/0069-1,S/N,1,2019-10-07 00:00:00,12.55,1.01,11.54,10,2019,0,NaN,NaN,1640863,NaN,NaN,3411,6935480,https://www.camara.leg.br/cota-parlamentar/doc...
125612,José Medeiros,4.244254e+10,204472.0,404.0,2019,MT,PODE,56,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,TECNEGOCIOS SOLUÇÕES EM INFORMÁTICA LTDA-ME,115.216.130/0019-0,007407,0,2020-09-25 00:00:00,599.00,0.00,599.00,9,2020,0,NaN,NaN,1720383,NaN,NaN,3411,7100291,https://www.camara.leg.br/cota-parlamentar/doc...
124782,Professora Rosa Neide,2.958637e+10,204467.0,408.0,2019,MT,PT,56,120,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,0,NaN,ATLANTA RENT A CAR,069.699.980/0011-3,1154,1,2020-11-17 01:00:00,2199.90,0.00,2199.90,10,2020,0,NaN,NaN,1728325,NaN,NaN,3312,7118977,https://www.camara.leg.br/cota-parlamentar/doc...
62637,Carlos Bezerra,8.349391e+08,141401.0,400.0,2019,MT,MDB,56,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - AZUL,092.962.950/0016-0,Bilhete: DJP4HX,0,2019-10-22 00:00:00,784.57,0.00,784.57,10,2019,0,Carlos Bezerra,BSB/CGB,0,0.0,NaN,1962,1711345,NaN


In [88]:
dados_mt.shape

(8960, 31)

### Exportando o dataset para o formato .csv:

In [91]:
dados_mt.to_csv('dados_mt.csv', index=False)